# Swarm access through VirES

> Abstract: VirES is a server/client architecture to help access time series of Swarm data and models. Access is enabled through a token generated on the website, and a Python package, viresclient, which provides the connection with the Python ecosystem (e.g. xarray).

In [ ]:
# Display important package versions used
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

**VirES** (Virtual environments for Earth Scientists) is a platform for data & model access, analysis, and visualisation for ESA's magnetic mission, **Swarm**

This tutorial introduces the Python interface to VirES, **`viresclient`**. We demonstrate usage of the primary Swarm magnetic dataset (`SW_OPER_MAGA_LR_1B`) and geomagnetic field models produced as part of the Swarm mission. Some knowledge of `pandas` and `matplotlib` is assumed.

Run this on the **[VRE](https://vre.vires.services/)** (Virtual Research Environment), where viresclient is already installed, or [check the instructions](https://viresclient.readthedocs.io/en/latest/installation.html) to set it up on your own Python environment.

For more information see:
- https://vires.services/ (Web interface)
- https://viresclient.readthedocs.io (Python interface)
- https://swarm.magneticearth.org (Guide to Virtual Research Environment / JupyterLab)
- https://earth.esa.int/eogateway/missions/swarm (Swarm mission)

## Configuration

In order to authenticate access to the VirES server, `viresclient` requires an access token - this ties communications between the server and the client to your account. **If you are using the VRE, this is handled automatically so you can skip this step.**

If you are running `viresclient` on a different machine, you will need to follow these instructions:

1. Create a user account at https://vires.services if you haven't already done so
2. Install viresclient in your Python environment - see https://viresclient.readthedocs.io/en/latest/installation.html
3. Create a new code cell here and execute the following:

   ```python
   from viresclient import set_token
   set_token("https://vires.services/ows", set_default=True)
   ```


4. You will now be directed to the [VirES token management page](https://vires.services/accounts/tokens/), and prompted to generate a new token and enter it here

Your access token should now have been saved to your environment and you won't need to provide it again. The token and its associated access URL are stored in a file: `~/.viresclient.ini` (this file can also be edited directly). You may generate and set a new token, or revoke old tokens, at any point. These are similar to passwords, so **should be kept secret** - if you accidentally leak a token, you can revoke it at the token management page and generate a new one. It is also possible to set access tokens via CLI. For more information, see https://viresclient.readthedocs.io/en/latest/config_details.html

To remove the configuration (assuming you left it in its default location), you can use the CLI command: `viresclient clear_credentials`

## Fetching some data

Import the `SwarmRequest` object which provides the VirES interface, and `datetime` which gives convenient time objects which can be used by `SwarmRequest.get_between()`

In [ ]:
from viresclient import SwarmRequest
import datetime as dt

The following code will fetch one day (i.e. around 15 orbits) of the scalar (`F`) measurements from Swarm Alpha. 

Several keyword arguments can be provided to `.set_products()` to specify the type of data you want. The `measurements` available depend on the collection chosen in `.set_collection()`. The same set of `auxiliaries` are available for all collections - here we also choose to fetch the `MLT` - magnetic local time.

`sampling_step="PT10S"` downsamples the data to 10 seconds, from the `MAGx_LR` default of 1 second. If no `sampling_step` is provided, the data will be accessed in its original form (i.e. here, 1-second sampling). These strings to choose the `sampling_step` should be provided as [ISO 8601 durations](https://en.wikipedia.org/wiki/ISO_8601#Durations) (e.g. `"PT1M"` for 1-minute sampling).

`start_time` and `end_time` in `.get_between()` together provide the time window you want to fetch data for - executing this line causes the request to be processed on the server and the data returned to you. NB: this returns data *up to but not including* `end_time`. Alternatively we can provide the start and end times as ISO_8601 strings.

In [ ]:
# Set up connection with server
request = SwarmRequest()
# Set collection to use
request.set_collection("SW_OPER_MAGA_LR_1B")
# Set mix of products to fetch:
#   measurements (variables from the given collection)
#   models (magnetic model predictions at spacecraft sampling points)
#   auxiliaries (variables available with any collection)
# Reference: https://viresclient.readthedocs.io/en/latest/available_parameters.html
# Also set additional configuration such as:
#   sampling_step
request.set_products(
    measurements=["F"],
    sampling_step="PT10S",
    auxiliaries=["MLT"]
)
# Fetch data from a given time interval
data = request.get_between(
    start_time=dt.datetime(2016,1,1),
    end_time=dt.datetime(2016,1,2)
)

The data is now contained within the object which we called `data`:

In [ ]:
data

The data is temporarily stored on disk and not yet loaded into memory - the `ReturnedData` object is actually a wrapper around a temporary CDF file which could be written to disk directly:

In [ ]:
# data.to_file("test_file.cdf", overwrite=True)

... but it is possible to directly transfer it to a `pandas.DataFrame` object:

In [ ]:
df = data.as_dataframe()
print(type(df))
df.head()

... or a `xarray.Dataset`:

In [ ]:
ds = data.as_xarray()
print(type(ds))
ds

Try plotting some things to visualise the data. The following shows the variation in field strength measured by the satellite as it passes between high and low latitudes, varying from one orbit to the next as it samples a different longitude.

In [ ]:
df.plot(y="F")
df.plot(y="F", x="Latitude")
df.plot(y="Latitude", x="Longitude")
df.plot(y="Latitude", x="Longitude", c="F", kind="scatter");

## Fetching model evaluations at the same time

Various (spherical harmonic) [models of the geomagnetic field](https://magneticearth.org/pages/models.html) are produced as Swarm mission products and these are available through VirES. They are evaluated on demand at the same points and times as the data sample points. Here we ask for the `MCO_SHA_2D` model, a dedicated core field model produced from Swarm data. By supplying `residuals=True` we will get the data-model residuals, named in the dataframe as `F_res_MCO_SHA_2D`.

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["F"],
    models=["MCO_SHA_2D"],
    residuals=True,
    sampling_step="PT10S"
)

data = request.get_between(
    start_time=dt.datetime(2016,1,1),
    end_time=dt.datetime(2016,1,2)
)

df = data.as_dataframe()
df.head()

The core field has been removed from the data so the amplitudes are much smaller. Can you interpret the new signals in terms of external fields, i.e. from the ionosphere and magnetosphere?

In [ ]:
df.plot(y="F_res_MCO_SHA_2D")
df.plot(y="F_res_MCO_SHA_2D", x="Latitude")
df.plot(y="Latitude", x="Longitude", c="F_res_MCO_SHA_2D", kind="scatter");

## More complex model handling

We can also remove a magnetospheric field model at the same time, by specifying a new model (which we call `MCO_MMA` here, but can be named whatever you like) which is the sum of core and magnetospheric models.

> See [Swarm_notebooks/notebooks/02b__viresclient-Available-Data](https://swarm.magneticearth.org/notebooks/02b__viresclient-available-data) for more examples of this - it is also possible to specify the spherical harmonic degrees (min/max) to use, and to provide your own `.shc` model.

The remaining signal is now primarily due to the ionosphere.

Note that here we are instead using the CI (comprehensive inversion) core and magnetosphere models (indicated by `2C` in the product names). The magnetospheric model (`MMA`) is split into two parts - "primary": external to the Earth, and "secondary": the induced counterpart internal to the Earth.

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["F"],
    models=["MCO_MMA = 'MCO_SHA_2C' + 'MMA_SHA_2C-Primary' + 'MMA_SHA_2C-Secondary'"],
    residuals=True,
    sampling_step="PT10S",
    auxiliaries=["MLT"]
)

data = request.get_between(
    start_time=dt.datetime(2016,1,1),
    end_time=dt.datetime(2016,1,2)
)

df = data.as_dataframe()
df.head()

In [ ]:
df.plot(y="F_res_MCO_MMA")
df.plot(y="F_res_MCO_MMA", x="Latitude")
df.plot(y="Latitude", x="Longitude", c="F_res_MCO_MMA", kind="scatter");